In [13]:
import numpy as np
import math
import operator

def dik(i,k):
    '''
    Input: i,k are the row and column. k: [2,n]
    Output: delay beween start of adjacent jobs i and k.
    '''
    # 
    assert i!=k
    d_max = 0
    # get index by -1
    i -= 1 
    k -= 1
    for j in range(M):
        if j == 0: 
            d_temp = sum(T[i][:j+1])
        else:
            d_temp = sum(T[i][:j+1])-sum(T[k][:j])
        # take the bigger dik
        if d_temp > d_max:
            d_max = d_temp
    return d_max

def MS(schedule):
    '''
    return the makespan of a schedule
    Input: Schedule should be a list
    Output: makespan as integer
    '''
    n = len(schedule)
    delay_sum = 0
    tn_sum = 0
    for i in range(n-1):
        delay_sum += dik(schedule[i],schedule[i+1])
    tn_sum = sum(T[schedule[-1]-1])
    Makespan = tn_sum+delay_sum
    return Makespan

def forward_shift(schedule):
    '''
    In: schedule from step 1
    Out: schedule after shift
    '''
    n = len(schedule)
    temp_schedule = [x for x in schedule]
    min_schedule = schedule
    min_ms = MS(min_schedule)
    
    for i in range(n-1):
        temp_schedule[i],temp_schedule[i+1] = temp_schedule[i+1],temp_schedule[i]
        print('{}th f_shift: {}, MS: {}'.format(i+1,temp_schedule, MS(temp_schedule)))

        if min_ms > MS(temp_schedule):
            min_ms = MS(temp_schedule)
            min_schedule = temp_schedule
        
    
    print('forward_shift optimal: {}, MS: {}\n'.format(min_schedule, min_ms))

    return min_schedule


def backward_shift(schedule):
    '''
    In: schedule from step 1
    Out: schedule after shift
    '''
    n = len(schedule)
    min_ms = math.inf
    min_schedule = []
    for i in range(n-1):
        pivot = [x for x in schedule if x==schedule[i+1]]
        rest = [x for x in schedule if not x==schedule[i+1]]
        temp_schedule = pivot+rest
        print('{}th b_shift: {}, MS: {}'.format(i+1,temp_schedule, MS(temp_schedule)))

        if min_ms > MS(temp_schedule):
            min_ms = MS(temp_schedule)
            min_schedule = temp_schedule
    print('backward_shift optimal: {}, MS: {}\n'.format(min_schedule, min_ms))
    return min_schedule
# x,y = backward_shift(tpt_sorted)
# print(x,y)

def k_pairs(schedule):
    n = len(schedule)
    if n % 2 == 0: # n is even
        k = int(n/2) # num of pairs
        # i =0
        temp_seq = partial_seq([schedule[0]], [schedule[1]])
        for i in range(1,k):
            print('-Check elements in pairs of 2:')
            temp_seq = partial_seq(temp_seq[:2*i], schedule[2*i : 2*i+2]) # check pairs
            print('-Check the 1st elements in pairs of 2:')
            # 1st element from new pairs
            temp_seq = partial_seq([x for x in temp_seq if x!=temp_seq[2*i]], [schedule[2*i]]) 
            print('-Check the 2st elements in pairs of 2:')
            #2nd element from new pairs
            temp_seq = partial_seq([x for x in temp_seq if x!=temp_seq[2*i+1]], [schedule[2*i+1]]) 
        
    elif n % 2 ==1: # n is odd
        k = math.ceil(n/2) # num of pairs, take the ceiling
        # i =0
        temp_seq = partial_seq([schedule[0]], [schedule[1]])
        for i in range(1,k-1): # ignore the half pair here
            print('-Check elements in pairs of 2:')
            temp_seq = partial_seq(temp_seq[:2*i], schedule[2*i : 2*i+2])
            print('-Check the 1st elements in pairs of 2:')
            temp_seq = partial_seq([x for x in temp_seq if x!=temp_seq[2*i]], [schedule[2*i]])
            print('-Check the 2st elements in pairs of 2:')
            temp_seq = partial_seq([x for x in temp_seq if x!=temp_seq[2*i+1]], [schedule[2*i+1]])
        # process the last single job
        temp_seq = partial_seq([x for x in temp_seq if x!=temp_seq[2*(k-1)]], [schedule[2*(k-1)]])
        
    return temp_seq
# k_pairs([2,5,6,1,3,4])

def partial_seq(schedule, obj):
    '''
    Insert object in to all possible place
    In: Partial Schedule and Object to insert, 
        object can be a int or sequence of int
    Out: Inserted partial sequence with min MS
    '''
    n = len(schedule)
    min_ms = math.inf
    min_schedule = []
    for i in range(n+1):
        temp_schedule = np.insert(schedule, i, obj)
        print(temp_schedule, MS(temp_schedule))
        if min_ms > MS(temp_schedule):
            min_ms = MS(temp_schedule)
            min_schedule = temp_schedule
    print('Optimal partial_seq & MS: {},{}\n'.format(min_schedule, min_ms))
    return min_schedule

def main():
    
    
    tpt_dict = {} # total processing time(tpt)
    for i in range(N): # sort jobs on tpt in descending order
        tpt_dict[i+1] = sum(T[i])
    tpt_sorted = [k for (k,v) in sorted(tpt_dict.items(), key=operator.itemgetter(1), reverse=True)]
    print('Sort jobs on descending total processing order:{}, MS:{}\n'.format(tpt_sorted,MS(tpt_sorted)))
    fws_seq = forward_shift(tpt_sorted)
    bws_seq = backward_shift(tpt_sorted)
    if MS(fws_seq) < MS(bws_seq):
        final_seq = k_pairs(fws_seq)
    elif MS(fws_seq) > MS(bws_seq):
        final_seq = k_pairs(bws_seq)
    elif MS(fws_seq) == MS(bws_seq):
        if MS(k_pairs(fws_seq)) > MS(k_pairs(bws_seq)):
            final_seq = k_pairs(bws_seq)
        elif MS(k_pairs(fws_seq)) < MS(k_pairs(bws_seq)):
            final_seq = k_pairs(fws_seq)
        else:
            final_seq = k_pairs(fws_seq)
    print('The final optimal schedule is : {}, MS: {}\n'.format(final_seq,MS(final_seq)))
                
        

if __name__ == '__main__':
    # step1 : Hard code the Time matrix
    # Tij is job i's process time on each machine
    T = np.array([[4,8,6,7],[3,4,7,8],[8,9,6,12],[10,16,11,13],[6,13,9,10],[9,14,10,9]])
    M = T.shape[1] # number of machines
    N = T.shape[0] # number of jobs
    main()

Sort jobs on descending total processing order:[4, 6, 5, 3, 1, 2], MS:99

1th f_shift: [6, 4, 5, 3, 1, 2], MS: 100
2th f_shift: [6, 5, 4, 3, 1, 2], MS: 103
3th f_shift: [6, 5, 3, 4, 1, 2], MS: 105
4th f_shift: [6, 5, 3, 1, 4, 2], MS: 111
5th f_shift: [6, 5, 3, 1, 2, 4], MS: 113
forward_shift optimal: [4, 6, 5, 3, 1, 2], MS: 99

1th b_shift: [6, 4, 5, 3, 1, 2], MS: 100
2th b_shift: [5, 4, 6, 3, 1, 2], MS: 95
3th b_shift: [3, 4, 6, 5, 1, 2], MS: 95
4th b_shift: [1, 4, 6, 5, 3, 2], MS: 96
5th b_shift: [2, 4, 6, 5, 3, 1], MS: 94
backward_shift optimal: [2, 4, 6, 5, 3, 1], MS: 94

[4 2] 58
[2 4] 53
Optimal partial_seq & MS: [2 4],53

-Check elements in pairs of 2:
[6 5 2 4] 94
[2 6 5 4] 79
[2 4 6 5] 75
Optimal partial_seq & MS: [2 4 6 5],75

-Check the 1st elements in pairs of 2:
[6 2 4 5] 91
[2 6 4 5] 76
[2 4 6 5] 75
[2 4 5 6] 77
Optimal partial_seq & MS: [2 4 6 5],75

-Check the 2st elements in pairs of 2:
[5 2 4 6] 86
[2 5 4 6] 71
[2 4 5 6] 77
[2 4 6 5] 75
Optimal partial_seq & MS: [2 5 

### Time matrix with Tij indicating process time of the i th job on the j th machine

In [14]:
T

array([[ 4,  8,  6,  7],
       [ 3,  4,  7,  8],
       [ 8,  9,  6, 12],
       [10, 16, 11, 13],
       [ 6, 13,  9, 10],
       [ 9, 14, 10,  9]])

### Enumerate all the possible permutation of jobs

In [16]:
import itertools
p = list(itertools.permutations([1,2,3,4,5,6]))
print('There are {} total possible job sequences.'.format(len(p)))

There are 720 total possible job sequences.


In [6]:
for i,l in enumerate(p):
    print(l,MS(l))

(1, 2, 3, 4, 5, 6) 101
(1, 2, 3, 4, 6, 5) 113
(1, 2, 3, 5, 4, 6) 128
(1, 2, 3, 5, 6, 4) 98
(1, 2, 3, 6, 4, 5) 128
(1, 2, 3, 6, 5, 4) 110
(1, 2, 4, 3, 5, 6) 99
(1, 2, 4, 3, 6, 5) 111
(1, 2, 4, 5, 3, 6) 128
(1, 2, 4, 5, 6, 3) 101
(1, 2, 4, 6, 3, 5) 128
(1, 2, 4, 6, 5, 3) 113
(1, 2, 5, 3, 4, 6) 113
(1, 2, 5, 3, 6, 4) 110
(1, 2, 5, 4, 3, 6) 111
(1, 2, 5, 4, 6, 3) 113
(1, 2, 5, 6, 3, 4) 83
(1, 2, 5, 6, 4, 3) 84
(1, 2, 6, 3, 4, 5) 113
(1, 2, 6, 3, 5, 4) 110
(1, 2, 6, 4, 3, 5) 111
(1, 2, 6, 4, 5, 3) 113
(1, 2, 6, 5, 3, 4) 95
(1, 2, 6, 5, 4, 3) 96
(1, 3, 2, 4, 5, 6) 99
(1, 3, 2, 4, 6, 5) 111
(1, 3, 2, 5, 4, 6) 111
(1, 3, 2, 5, 6, 4) 81
(1, 3, 2, 6, 4, 5) 111
(1, 3, 2, 6, 5, 4) 93
(1, 3, 4, 2, 5, 6) 85
(1, 3, 4, 2, 6, 5) 97
(1, 3, 4, 5, 2, 6) 104
(1, 3, 4, 5, 6, 2) 96
(1, 3, 4, 6, 2, 5) 108
(1, 3, 4, 6, 5, 2) 104
(1, 3, 5, 2, 4, 6) 119
(1, 3, 5, 2, 6, 4) 101
(1, 3, 5, 4, 2, 6) 112
(1, 3, 5, 4, 6, 2) 123
(1, 3, 5, 6, 2, 4) 93
(1, 3, 5, 6, 4, 2) 100
(1, 3, 6, 2, 4, 5) 123
(1, 3, 6, 2, 5, 4) 105
(

(4, 6, 3, 1, 2, 5) 112
(4, 6, 3, 1, 5, 2) 115
(4, 6, 3, 2, 1, 5) 119
(4, 6, 3, 2, 5, 1) 106
(4, 6, 3, 5, 1, 2) 124
(4, 6, 3, 5, 2, 1) 114
(4, 6, 5, 1, 2, 3) 109
(4, 6, 5, 1, 3, 2) 107
(4, 6, 5, 2, 1, 3) 100
(4, 6, 5, 2, 3, 1) 102
(4, 6, 5, 3, 1, 2) 112
(4, 6, 5, 3, 2, 1) 106
(5, 1, 2, 3, 4, 6) 125
(5, 1, 2, 3, 6, 4) 122
(5, 1, 2, 4, 3, 6) 123
(5, 1, 2, 4, 6, 3) 125
(5, 1, 2, 6, 3, 4) 107
(5, 1, 2, 6, 4, 3) 108
(5, 1, 3, 2, 4, 6) 123
(5, 1, 3, 2, 6, 4) 105
(5, 1, 3, 4, 2, 6) 109
(5, 1, 3, 4, 6, 2) 120
(5, 1, 3, 6, 2, 4) 117
(5, 1, 3, 6, 4, 2) 124
(5, 1, 4, 2, 3, 6) 123
(5, 1, 4, 2, 6, 3) 108
(5, 1, 4, 3, 2, 6) 105
(5, 1, 4, 3, 6, 2) 117
(5, 1, 4, 6, 2, 3) 119
(5, 1, 4, 6, 3, 2) 122
(5, 1, 6, 2, 3, 4) 114
(5, 1, 6, 2, 4, 3) 115
(5, 1, 6, 3, 2, 4) 117
(5, 1, 6, 3, 4, 2) 121
(5, 1, 6, 4, 2, 3) 121
(5, 1, 6, 4, 3, 2) 118
(5, 2, 1, 3, 4, 6) 116
(5, 2, 1, 3, 6, 4) 113
(5, 2, 1, 4, 3, 6) 113
(5, 2, 1, 4, 6, 3) 115
(5, 2, 1, 6, 3, 4) 110
(5, 2, 1, 6, 4, 3) 111
(5, 2, 3, 1, 4, 6) 118
(5, 2, 3, 1